In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from numpy import linalg as LA
import pandas as pd
import itertools  
from tqdm import tqdm

from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from scipy.integrate import odeint
from scipy.optimize import minimize
from scipy.optimize import Bounds

In [ ]:
def d_dzeta_cent_O6(func, ts):
    delta_z = ts[1] - ts[0]
    f_res = -1*func[:-6] + 9*func[1:-5] - 45*func[2:-4] + 0\
            + 45*func[4:-2] - 9*func[5:-1] + 1*func[6:]
    return f_res/(60*delta_z)

def d_ddzeta_cent_O6(func, ts):
    delta_z = ts[1] - ts[0]
    f_res = 2*func[:-6] - 27*func[1:-5] + 270*func[2:-4] - 490*func[3:-3]\
            + 270*func[4:-2] - 27*func[5:-1] + 2*func[6:]
    return f_res/(180*delta_z**2)

def specific_value(func, val, modif_r):
    specific_range = np.where(np.logical_and(func>=val-modif_r, func<=val+modif_r))
    return specific_range[0]

In [ ]:
def permutation_list(gamma, num):
    gamma_comb = []
    gamma_list = list(itertools.combinations(gamma,num))
    for gam_one in gamma_list:
        temp_gamma = 1
        for gam in gam_one:
            temp_gamma = temp_gamma * gam
        gamma_comb.append(temp_gamma)
    return gamma_comb

def F_u(u, gamma):
    k = len(gamma)
    F_u_result = u
    for i in range(1,k+2):
        F_u_result = F_u_result + \
              (-1)**i * (np.sum(permutation_list(gamma, i-1)) + \
                             np.sum(permutation_list(gamma, i)))*u**(i+1)
    return F_u_result

def dF_u(u, gamma):
    k = len(gamma)
    F_u_result = 1
    for i in range(1,k+2):
        F_u_result = F_u_result + \
              (i+1)*(-1)**i * (np.sum(permutation_list(gamma, i-1)) + \
                             np.sum(permutation_list(gamma, i)))*u**i
    return F_u_result

In [ ]:
# Type 1 inverse problem
def du_dx_u_q(U, x, eps, V, gamma):
    F_u = U[0] * (1-U[0]) 
    for gam in gamma:
        F_u = F_u * (1-gam*U[0])
    left_1 = (eps*V*F_u - U[1])/(1 - eps*V**2)
    left_2 = (left_1 + U[1])/(eps*V)
    return [left_1, left_2]

def du_dx_u_q_eps_zero(U, x, eps, V, gamma):
    F_u = U[0] * (1-U[0]) 
    for gam in gamma:
        F_u = F_u * (1-gam*U[0])
    return [-U[1], F_u - V*U[1]]

# Type 2 inverse problem 
def du_dx(U, x, eps, V, gamma):
    F_u = U[0] * (1-U[0]) 
    for gam in gamma:
        F_u = F_u * (1-gam*U[0])
        
    dF_du = F_u/U[0] - F_u/(1-U[0])
    for gam in gamma:
        dF_du = dF_du - gam*F_u / (1-gam*U[0])
    return [U[1], ((1- eps*dF_du) * V*U[1] + F_u)/(eps*V**2-1)]

# Generate ODE output
def ODE_UQ(tiny, gamma, V, eps, t_end):
    U0 = [1-tiny, -tiny]
    ts = np.linspace(0, t_end, 20000)
    if eps == 0:
        Us = odeint(du_dx_u_q_eps_zero, U0, ts, args=(eps,  V, gamma))
    else:
        Us = odeint(du_dx_u_q, U0, ts, args=(eps,  V, gamma))
    u_res = Us[:,0]
    q_res = Us[:,1]
    return [u_res, q_res, ts]

def ODE_U(tiny, gamma, V, eps, t_end):
    U0 = [1-tiny, -tiny]
    ts = np.linspace(0, t_end, 20000)
    Us = odeint(du_dx, U0, ts, args=(eps,  V, gamma))
    u_res = Us[:,0]
    du_res = Us[:,1]
    return [u_res, du_res, ts]

def U_Q_output(gamma, V, eps, t_end):
    tiny = 1E-6
    u_res, q_res, ts = ODE_UQ(tiny, gamma, V, eps, t_end)
    z_upon = specific_value(u_res, 0.95, 0.01)[0]
    z_low = specific_value(u_res, 0.05, 0.01)[-1]
    u_res_spec = u_res[z_upon:z_low]
    q_res_spec = q_res[z_upon:z_low]
    u_dz = d_dzeta_cent_O6(u_res_spec, ts)
    q_dz = d_dzeta_cent_O6(q_res_spec, ts)
    u_spec = u_res_spec[3:-3]
    q_spec = q_res_spec[3:-3]
    t_spec = ts[z_upon:z_low]
    t_spec = t_spec[3:-3]
    return u_spec, q_spec, u_dz, q_dz, t_spec

def U_output(gamma, V, eps, t_end):
    tiny = 1E-6
    u_res, q_res, ts = ODE_U(tiny, gamma, V, eps, t_end)
    z_upon = specific_value(u_res, 0.95, 0.01)[0]
    z_low = specific_value(u_res, 0.05, 0.01)[-1]
    u_res_spec = u_res[z_upon:z_low]
    u_dz = d_dzeta_cent_O6(u_res_spec, ts)
    u_ddz = d_ddzeta_cent_O6(u_res_spec, ts)
    u_spec = u_res_spec[3:-3]
    t_spec = ts[z_upon:z_low]
    t_spec = t_spec[3:-3]
    return u_spec, u_dz, u_ddz, t_spec

In [ ]:
# Two types of optimization problem
# Type 1
def optimize_gam_UQ(para, u, du, dq, lam):
    V = para[0]
    gamma = para[1:]
    F_u_res = F_u(u, gamma)
    right = F_u_res + V*du
    zero_norm = np.sum(gamma != 0)
    return LA.norm(right - dq) + lam * zero_norm

# Type 2
def optimize_U(para, u, du, ddu, lam):
    eps = para[0]
    V = para[1]
    gamma = para[2:]
    F_u_res = F_u(u, gamma)
    dF_u_res = dF_u(u, gamma)
    zero_norm = np.sum(gamma != 0)
    return LA.norm(ddu*(eps*V**2 - 1) + du*V * (eps*dF_u_res - 1) - F_u_res) + lam * zero_norm

def optimize_U_eps0(para, u, du, ddu, lam):
    V = para[0]
    gamma = para[1:]
    F_u_res = F_u(u, gamma)
    zero_norm = np.sum(gamma != 0)
    return LA.norm(-ddu - du*V- F_u_res) + lam * zero_norm

In [ ]:
# Main process of parameter estimation
# Type 1, k is the number of Gamma
def para_solver_UQ(u, du, dq, k, v_max):
    gam0 = np.linspace(1,0,k)
    para0 = np.append(v_max, gam0)
    lam = 1e-8
    bounds = Bounds(np.append(2, np.zeros(k)), np.append(v_max, np.ones(k)))
    for i in range(3):
        min_res = minimize(optimize_gam_UQ,  para0, args=(u, du , dq, lam), 
                      method='L-BFGS-B', bounds = bounds, tol=1e-16)
        para0 = np.append(min_res.x[0], gam0)
    return min_res

def eps_solver_UQ(du, q, dq, V0):
    return np.mean((du + q) / (V0*dq))

# Type 2
def para_solver_U(u, du, ddu, k, v_max):
    gam0 = np.linspace(1,0,k)
    lam = 1e-8
    para0 = np.append(v_max, gam0)
    bounds = Bounds(np.append(2, np.zeros(k)), np.append(v_max, np.ones(k)))
    for j in range(3):
        min_res_ini = minimize(optimize_U_eps0,  para0, args=(u, du , ddu, lam), 
                          method='L-BFGS-B', bounds = bounds, tol=1e-16)
        para0 = np.append(min_res_ini.x[0], gam0)
        if para0[0] < 2:
            para0[0] = 2
        elif para0[0] > v_max:
            para0[0] = v_max
    para0 = np.append(0, [min_res_ini.x])
    bounds = Bounds(np.append([0, 2], np.zeros(k)), np.append([0.25, v_max], np.ones(k)))

    for i in range(2):
        min_res = minimize(optimize_U,  para0, args=(u, du , ddu, lam), 
                      method='L-BFGS-B', bounds = bounds, tol=1e-16)
        para0 = np.append([0, min_res.x[1]], gam0)
    return min_res

In [ ]:
V_list = np.linspace(2, 5, 4)

eps_amount = 2
gamma_1 = np.linspace(0, 0.65, 3)
gamma_2 = np.linspace(0, 0.55, 2)
gamma_3 = np.linspace(0, 0.45, 2)

Test_para = {'V':[], 'eps':[], 'gamma':[]}
performance_UQ_case = {'V':[], 'eps':[], 'gamma':[],'error':[]}
performance_U_case= {'V':[], 'eps':[], 'gamma':[],'error':[]}

# Set the range of zeta
ts_end = [350, 500, 650, 800]

In [ ]:
for V in V_list:
    eps_list = np.linspace(0,1/V**2, eps_amount+1)[:-1]
    for eps in eps_list:
        for gamma1 in gamma_1:
            for gamma2 in gamma_2:
                for gamma3 in gamma_3:
                    # Add 0 is easy to calculate RMSE
                    gamma = [gamma1, gamma2, gamma3,0,0,0]
                    Test_para['V'].append(V)
                    Test_para['eps'].append(eps)
                    Test_para['gamma'].append(np.sort(gamma)[::-1])

In [ ]:
for i in tqdm(range(len(Test_para['V']))):
    V = Test_para['V'][i]
    eps = Test_para['eps'][i]
    gamma = Test_para['gamma'][i]
    t_end = ts_end[int(V)-2]
    u, q, u_dz, q_dz,_ = U_Q_output(gamma, V, eps, t_end)
    u_para = para_solver_UQ(u, u_dz, q_dz, 6, 500)
    eps_p = eps_solver_UQ(u_dz, q, q_dz, u_para.x[0])

    performance_UQ_case['V'].append(u_para.x[0])
    performance_UQ_case['eps'].append(eps_p)
    performance_UQ_case['gamma'].append(np.sort(u_para.x[1:])[::-1])
    performance_UQ_case['error'].append(u_para.fun)

    u, u_dz, u_ddz,_ = U_output(gamma, V, eps, t_end)
    u_para = para_solver_U(u, u_dz, u_ddz, 6, 500)

    performance_U_case['V'].append(u_para.x[1])
    performance_U_case['eps'].append(u_para.x[0])
    performance_U_case['gamma'].append(np.sort(u_para.x[2:])[::-1])
    performance_U_case['error'].append(u_para.fun)

100%|██████████| 96/96 [12:34<00:00,  7.86s/it]


In [ ]:
print('Type 1 inverse problem RMSE for V:',mean_squared_error(Test_para['V'], performance_UQ_case['V'], squared=False))
print('Type 1 inverse problem RMSE for epsilon:',mean_squared_error(Test_para['eps'], performance_UQ_case['eps'], squared=False))
print('Type 1 inverse problem RMSE for all gamma:',mean_squared_error(Test_para['gamma'], performance_UQ_case['gamma'], squared=False))

Type 1 inverse problem RMSE for V: 9.44203656739451e-06
Type 1 inverse problem RMSE for epsilon: 1.7921930718217773e-05
Type 1 inverse problem RMSE for all gamma: 0.0023370927756253836


In [ ]:
print('Type 2 inverse problem RMSE for V:',mean_squared_error(Test_para['V'], performance_U_case['V'], squared=False))
print('Type 2 inverse problem RMSE for epsilon:',mean_squared_error(Test_para['eps'], performance_U_case['eps'], squared=False))
print('Type 2 inverse problem RMSE for all gamma:',mean_squared_error(Test_para['gamma'], performance_U_case['gamma'], squared=False))

Type 2 inverse problem RMSE for V: 2.1938258947601516e-05
Type 2 inverse problem RMSE for epsilon: 0.0001538247850870462
Type 2 inverse problem RMSE for all gamma: 0.0044348934379889685
